In [6]:
!pip install gradio spacy -q
!python -m spacy download uk_core_news_sm -q

print("Бібліотеки встановлено, модель завантажено!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 81.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('uk_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Бібліотеки встановлено, модель завантажено!


In [7]:
import spacy
from spacy import displacy
import warnings

warnings.filterwarnings("ignore")

class NamedEntityRecognizer:
    def __init__(self, model_name="uk_core_news_sm"):
        """Ініціалізація моделі NLP."""
        try:
            self.nlp = spacy.load(model_name)
        except OSError:
            import subprocess
            subprocess.run(["python", "-m", "spacy", "download", model_name])
            self.nlp = spacy.load(model_name)

    def extract_entities(self, text: str):
        """
        Повертає список знайдених сутностей та HTML візуалізацію.
        Алгоритм: Використання нейромережевої моделі spaCy для контекстного аналізу.
        """
        if not text or not isinstance(text, str):
            raise ValueError("Вхідні дані мають бути непорожнім рядком.")

        doc = self.nlp(text)

        entities_list = [
            (ent.text, ent.label_, spacy.explain(ent.label_))
            for ent in doc.ents
        ]

        html = displacy.render(doc, style="ent", page=True)

        return entities_list, html

recognizer = NamedEntityRecognizer()
print("Модуль ініціалізовано успішно.")

Модуль ініціалізовано успішно.


In [8]:
import unittest

class TestNER(unittest.TestCase):
    def setUp(self):
        self.ner = NamedEntityRecognizer()

        self.text_1 = "Тарас Шевченко народився в селі Моринці."
        self.text_2 = "Компанія Google відкрила офіс у Києві."
        self.text_3 = "Президент Зеленський зустрівся з лідерами ЄС."

    def test_person_extraction(self):
        """Перевірка чи знаходить ім'я Тарас Шевченко"""
        entities, _ = self.ner.extract_entities(self.text_1)
        found_names = [e[0] for e in entities if e[1] == 'PER']
        self.assertIn("Тарас Шевченко", found_names)

    def test_location_extraction(self):
        """Перевірка чи знаходить локацію Київ"""
        entities, _ = self.ner.extract_entities(self.text_2)
        found_locs = [e[0] for e in entities if e[1] == 'LOC']
        self.assertIn("Києві", found_locs)

    def test_org_extraction(self):
        """Перевірка чи знаходить організацію ЄС"""
        entities, _ = self.ner.extract_entities(self.text_3)
        found_orgs = [e[0] for e in entities if e[1] == 'ORG']
        self.assertIn("ЄС", found_orgs)

    def test_empty_input(self):
        """Перевірка обробки помилок"""
        with self.assertRaises(ValueError):
            self.ner.extract_entities("")

unittest.main(argv=[''], verbosity=2, exit=False)

test_empty_input (__main__.TestNER.test_empty_input)
Перевірка обробки помилок ... ok
test_location_extraction (__main__.TestNER.test_location_extraction)
Перевірка чи знаходить локацію Київ ... 

ok
test_org_extraction (__main__.TestNER.test_org_extraction)
Перевірка чи знаходить організацію ЄС ... 

ok
test_person_extraction (__main__.TestNER.test_person_extraction)
Перевірка чи знаходить ім'я Тарас Шевченко ... 

ok

----------------------------------------------------------------------
Ran 4 tests in 16.759s

OK


In [10]:
import gradio as gr

def process_text(text):
    """Обгортка для Gradio з обробкою помилок."""
    try:
        entities, html_visual = recognizer.extract_entities(text)

        if not entities:
            return "Сутностей не знайдено.", html_visual

        result_text = "Знайдені власні назви:\n"
        for name, label, desc in entities:
            result_text += f"- {name} ({label}): {desc}\n"

        return result_text, html_visual

    except ValueError as ve:
        return f"Помилка вводу: {str(ve)}", ""
    except Exception as e:
        return f"Критична помилка: {str(e)}", ""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🇺🇦 Лабораторна робота: Визначення власних назв
        **Тема:** Комп'ютерна лінгвістика.
        **Алгоритм:** Named Entity Recognition (NER) на базі моделі spaCy `uk_core_news_sm`.
        """
    )

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Введіть текст українською мовою",
                placeholder="Наприклад: Леся Українка відвідала Єгипет у 1909 році.",
                lines=5
            )
            analyze_btn = gr.Button("🔍 Аналізувати", variant="primary")

            gr.Markdown("### Приклади для тестування:")
            gr.Examples(
                examples=[
                    ["Ілон Маск заявив про нові плани SpaceX щодо польоту на Марс."],
                    ["Верховна Рада України ухвалила новий бюджет."],
                    ["Карпати - це неймовірне місце для відпочинку взимку."]
                ],
                inputs=input_text
            )

        with gr.Column():
            output_html = gr.HTML(label="Візуалізація")
            output_list = gr.Textbox(label="Список знайдених сутностей", lines=5)

    analyze_btn.click(
        fn=process_text,
        inputs=input_text,
        outputs=[output_list, output_html]
    )


demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d4b68cbcd0a1845363.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d4b68cbcd0a1845363.gradio.live
